## 데이터 불러오기

In [ ]:
import pandas as pd
test = pd.read_csv('/content/news_test.csv', encoding = 'utf-8-sig')
sub=pd.read_csv('/content/sample_submission.csv',encoding='utf-8')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install kobert-transformers

## 시간 측정 시작

In [ ]:
import time
start = time.time()

## Library  불러오기

In [ ]:
import tensorflow as tf

from transformers import BertTokenizer, TFBertModel
from transformers import BertConfig, BertModel, BertForPreTraining
import numpy as np
from tqdm.notebook import tqdm
import os
import re
from kobert_transformers import get_tokenizer

## pos_Tagger, Tokenizer, pretraind_embedding, Model 불러오기

In [ ]:
# tokenizer
plus_vocab=pd.read_csv('/content/얼그레이/1. Data/plus 1500 vocab.csv',encoding='utf-8')
tokenizer = get_tokenizer()
tokenizer.add_tokens(plus_vocab['0'][:500].tolist())

# model 불러오기 및 input data 설정
SEQ_LEN  = 50
model = TFBertModel.from_pretrained("monologg/kobert", from_pt=True)
model.resize_token_embeddings(len(tokenizer))

token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids')
mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks')
segment_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_segment')

bert_outputs = model([token_inputs, mask_inputs, segment_inputs])
bert_outputs = bert_outputs[1]

model_drop = tf.keras.layers.Dropout(0.5)(bert_outputs)
model_first = tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02))(model_drop)
best_model = tf.keras.Model([token_inputs, mask_inputs, segment_inputs], model_first)

best_model.load_weights('/content/얼그레이/5. Model/model3-010-0.993600.h5')

All PyTorch model weights were used when initializing TFBertModel.

All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7f0ee4967e58> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7f0ee4967e58> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


## 형태소 분석 + 전처리

In [ ]:
# preprocessing
def alpha_num(text):
  return re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\]‘|\(\)\[\]\<\>`\'…》;·_ ~{}°˚？＃∼＇]', '', text)
test['title2'] = test['title'].apply(alpha_num)
test['content2'] = test['content'].apply(alpha_num)

# Data
def convert_data(data_df):
    global tokenizer
    
    SEQ_LEN1 = 20 #SEQ_LEN : 버트에 들어갈 인풋의 길이
    SEQ_LEN2 = 30
    tokens, masks, segments, targets = [], [], [], []
    for i in tqdm(range(len(data_df))):
        # token : 문장을 토큰화함
        token1 = tokenizer.encode(data_df[DATA_COLUMN][i], max_length=SEQ_LEN1, truncation=True, padding='max_length')
        token2 = tokenizer.encode(data_df[DATA_COLUMN2][i], max_length=SEQ_LEN2, truncation=True, padding='max_length')
        token=token1+token2

        # 마스크는 토큰화한 문장에서 패딩이 아닌 부분은 1, 패딩인 부분은 0으로 통일
        num_zeros = token1.count(0)
        mask1 = [1]*(SEQ_LEN1-num_zeros) + [0]*num_zeros
        num_zeros = token2.count(0)
        mask2 = [1]*(SEQ_LEN2-num_zeros) + [0]*num_zeros 
        mask=mask1+mask2      
      
        segment=[0]*SEQ_LEN1+[1]*SEQ_LEN2
        # 버트 인풋으로 들어가는 token, mask, segment를 tokens, segments에 각각 저장
        tokens.append(token)
        masks.append(mask)
        segments.append(segment)
 
    # tokens, masks, segments, 정답 변수 targets를 numpy array로 지정    
    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
 
    return [tokens, masks, segments]
 
# 위에 정의한 convert_data 함수를 불러오는 함수를 정의
def load_data(pandas_dataframe):
    data_df = pandas_dataframe
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_x= convert_data(data_df)
    return data_x

DATA_COLUMN = "title2"
DATA_COLUMN2= "content2"

# model input에 맞게 test data 만들기
test_x = load_data(test)

KeyboardInterrupt: ignored

## 예측

In [ ]:
# prediction
pred=best_model.predict(test_x, batch_size=128)

# 예측한 확률이 0.5보다 크면 1, 0.5보다 작으면 0
pred_01=[]
for i in range(len(pred)):
  pred_01.append(1 if pred[i] >=0.5 else 0)
sub['info']=pred_01

In [ ]:
print(time.time() - start)

# 제출

In [ ]:
sub.to_csv('/content/submission.csv')